# Generate Org Effectiveness Dashboard Data
Generate the data for updating the Org Effectiveness Dashboard

## Functions
- Generate Labour Costs (actual) for most recent month
- Generate Labour Costs (forecast) for remaining months in year
- Combine with FTE (actual for most recent month and forecast data)

## Source files
- Labour Cost report (from Finance with one month Actual and remaining months forecasts)
- FTE Summary Data 
    - This will be for the most recent Budget Wave
    - Generated by the Budget Pack Summaries script

### Preparation
- Labour Cost file
    - Remove top three rows
    - Convert number columns to Number (remove commas)
    - Delete Grand total row
    - Review fact column names
        -  Should start with Month and end with Year
    - Save as a CSV file with the name Labour_cost_full_Month_Year.csv
- Current fte_summary_data.csv file
    - Save as a CSV file with the name fte_summary_data_full_Month_Year.csv
- Run the Labour Costs file through the Cost Center Audits script to make sure all JDE Business Units have been mapped to a Workday Business Unit
    - Address any that have not been mapped and update the cost_center_mapping.csv file

### Processing
- Update month_year variable with the month being processed (most recent actuals)
    - e.g. 'October_2025'
- Restart the Kernal and run the notebook
- Address any JDE Business Units that appear as Unknown (will not have been mapped)
- When happy with the output copy the data columns and paste into the data sheet of the Organisational Effectiveness Dashboard
    - Make sure the columns are correct and the data goes against the correct rows (month_code and business_unit)
- Hide the data sheet
- Refresh the data
- Save and upload

In [1]:
import sys
import pandas as pd
import pyarrow as pa
string_pa = pd.ArrowDtype(pa.string())
float_pa = pd.ArrowDtype(pa.float64())
int8_pa = pd.ArrowDtype(pa.int8())

In [2]:
# Functions
def add_wd_bu(df, mapping, month_year):
    """Add the Workday Business Unit to labour data.

    Args:
        df (df): Labour data
        maping (df): Mapping data
        month_year (str): Month and Year being reported on.

    Returns:
        updated_df (df): Labour data with Workday Business Unit added.
    """
    bu_mapping = mapping.loc[:,['business_unit_jde', 'business_unit_wd']]
    updated_df = (df
                  .merge(bu_mapping, on='business_unit_jde', how='left')
                 )

    # Fill missing actual and budget with 0
    updated_df = (updated_df
                  .assign(**updated_df
                          .select_dtypes(include='number')
                          .fillna(0)
                         )
                 )
    
    # Fill missing business units with Unknown so get retained
    updated_df['business_unit_wd'] = updated_df.business_unit_wd.fillna('Unknown')

    # Check for Unknown Business Units
    process_unknown_bus(updated_df, f'Labour_Costs_{month_year}')

    # Move WD BU to start and drop JDE BU
    data_cols = list(updated_df.columns)[1:-1] # Drop first and last column
    reordered_cols = ['business_unit_wd']
    reordered_cols.extend(data_cols)
    
    return updated_df.loc[:, reordered_cols]    


def clean_cols(df):
    """Clean columns.

    Lower column names and replace spaces with underscores.

    Args:
        df (df): DataFrame to clean.

    Returns:
        cleaned_df (df): DataFrame with renamed columns.
    """
    return (df.rename(columns=lambda x: x.lower()
                      .replace('\n', ' ')
                      .replace(' ', '_')
                      .replace('/', '_')
                     )
           )


def gen_labour_cols(report_month):
    """Generate list of labour columns to keep.

    Generates a list of columns to keep from the labour report. 
    This will be the dimension columns required and the data columns 
    for the actual month and the remaining forecast months.

    Args:
        report_month (str): Month_Year.

    Returns:
        labour_cols (list): Columns to be retained.
    """
    month, year = gen_month_year(report_month)
    month_position = get_month_position(month)
    
    month_list = gen_month_list()
    
    # Get actuals month column name
    actual_month_columns = gen_month_cols_list(month_list, year, 'actual')
    actual_month_col = actual_month_columns[month_position]

    # Get forecast months column names
    forecast_month_columns = gen_month_cols_list(month_list, year, 'forecast')
    forecast_month_cols = forecast_month_columns[month_position+1:] # All columns after actual month

    # Generate list of columns
    labour_cols = ['business_unit_jde', actual_month_col]
    labour_cols.extend(forecast_month_cols)

    return labour_cols    


def gen_labour_cost_df(df):
    """Generate DataFrame with Labour Costs.

    Generates a DataFrame with the Actual Labour Costs per Workday 
    Business Unit for the most recent report month and forecast 
    costs for each month remaining in the year. Each month is 
    placed into a single Labour Costs column.

    Args:
        df (df): Labour Costs data.

    Returns:
        labour_df (df): Labour costs data in one column.
    """
    data_cols = {'month_code': pd.Series(dtype=string_pa),
                 'business_unit_wd': pd.Series(dtype=string_pa),
                 'labour_cost': pd.Series(dtype=float_pa)}
    labour_cost_df = pd.DataFrame(data_cols)

    # Move each data column into the labour_cost column
    for col in list(df.columns)[1:]:
        month_, year_ = gen_month_year(col)
        try:
            month_ = month_to_num(month_)
        except KeyError as e:
           raise KeyError(e)
        month_code_ = gen_month_code(month_, year_)
        
        df_ = df.loc[:,['business_unit_wd', col]]
        df_['month_code'] = month_code_
        df_ = df_.rename(columns={col: 'labour_cost'})
        df_ = df_.loc[:, ['month_code', 'business_unit_wd', 'labour_cost']]
        
        labour_cost_df = pd.concat([labour_cost_df, df_], ignore_index=True)
    
    # Group data
    labour_cost_df = (labour_cost_df
                      .groupby(['month_code', 'business_unit_wd'], as_index=False).sum()
                      .assign(month_code=lambda x: x.month_code.astype(string_pa))
                      .rename(columns={'business_unit_wd': 'business_unit'})
                      .sort_values(['month_code', 'business_unit'])
                      .query('business_unit != "Unknown"')
                     )
    return labour_cost_df
    
    
def gen_month_code(month, year):
    """Generate a month code from a month and a year.

    Args:
        month (str): Month code from 01 to 12
        year (str): Four digit year.

    Returns:
        month_code (str): YYYYMM.
    """
    return f'{year}{month}'


def gen_month_cols_list(months, year, metric):
    """Generate a list of month column headings.

    Generates a list of month column headings in the format 
    'january_actual_nzd_2025' where metric is actual and year 
    is 2025.

    Args:
        months (list): List of months to include
        year (str): Year to include
        metric (str): Metric to include in column name.

    Returns:
        month_cols_list (list): Each month as a column heading.
    """
    # Add _nzd to forecast months
    if metric == 'forecast':
        metric = f'{metric}_nzd'
    return [f'{month}_{metric}_{year}' for month in months]


def gen_month_list():
    """Generate a list of months from Janaury to December.

    Returns:
        months (list): List of months from January to December.
    """
    months = ['january', 'february', 'march', 'april', 'may',
              'june', 'july', 'august', 'september', 'october',
              'november', 'december']
    return months


def gen_month_year(report_month):
    """Get report month and year.

    report_month needs to be a string that is separated using underscores.
    The month component must be at the start of the string and the 
    year component at the end of the string.

    Args:
        report_month (str): Month_Year.

    Returns:
        month (str): Month name
        year (str): Four digit year.
    """
    month = f'{report_month.split("_")[0]}'
    year = f'{report_month.split("_")[-1]}'
    return (month, year)    


def get_month_position(month):
    """Get position of month.

    Month position will be used to extract the month from 
    a list of months (January to December) accounting for 0-based 
    indexing.

    Args:
        month (str): Name of month,

    Returns:
        month_position (int): Position of month in month list.
    """
    month_positions = {'january': 0, 'february': 1, 'march': 2, 'april': 3, 'may': 4,
                       'june': 5, 'july': 6, 'august': 7, 'september': 8,
                       'october': 9, 'november': 10, 'december': 11}
    month_position = month_positions.get(month.lower())
    return month_position    


def month_to_num(month):
    """Converts month name to month number.

    Number will be a string in the format 01 for January and 
    12 for December.

    Args:
        month (str): Month name.

    Returns:
        month_num (str): Month number.
    """
    month_codes = {'january': '01', 'february': '02', 'march': '03', 'april': '04',
                   'may': '05', 'june': '06', 'july': '07', 'august': '08',
                   'september': '09', 'october': '10', 'november': '11',
                   'december': '12'}

    try:
        return month_codes.get(month.lower())
    except KeyError:
        raise KeyError(f'Unable to get month code for {month}')
    
    
    
def process_unknown_bus(df, file_end):
    """Save a file with any unknown Business Units.

    Args:
        df (df): Output data to process
        file_end (str): To place at end of file name.
    """
    missing_bu = df.query('business_unit_wd == "Unknown"').copy()
    
    if missing_bu.empty:
        return
   
    f_name = f'Troubleshooting/Missing_Business_Unit_{file_end}.xlsx'
    print('There are entries that do not have a Business Unit attached. Please check the file')
    print(f'File has been saved to {f_name}')
    missing_bu.to_excel(f_name, index=False)
    return


def tweak_cc_mapping(df):
    """Clean Cost Center mapping dataset.

    Set dtypes.
    Reduces columns.
    
    Args:
        df (df): Labour cost data.

    Returns:
        tweaked_df (df): Cleaned Labour cost data.
    """
    tweaked_df = (df
                  .assign(business_unit_jde=lambda x: x.business_unit_jde.astype(string_pa),
                          cost_center_code_wd=lambda x: x.cost_center_code_wd.astype(string_pa)
                         )
                  .loc[:,['cost_center_jde', 'business_unit_jde', 'cost_center_wd', 'cost_center_code_wd',
                          'business_unit_wd']]
                 )
    return tweaked_df

    
def tweak_fte(df, report_month):
    """Clean FTE dataset.

    Rename columns.
    Set dtypes.
    Remove unwanted columns.
    Reduce to most recent actual and all forecast months.
    Remove Global entries.

    Note: headcount and num_leaders columns are empty but in 
    future update will be populated.
    
    Args:
        df (df): FTE data
        report_month (str): Month_Year.

    Returns:
        tweaked_df (df): Cleaned FTE data.    
    """
    # Set up filtering to keep only most recent actuals onwards
    month, year = gen_month_year(report_month)
    month = month_to_num(month)
    min_month_code = int(gen_month_code(month, year))
    
    keep_cols = ['month_code', 'business_unit', 'fte', 'fte_budget', 'avg_headcount',
                 'voluntary_leavers', 'involuntary_leavers', 'total_leavers',
                 '12_month_rolling_turnover', 'open_positions_fte', 'filled_positions_fte',
                 'headcount', 'num_leaders']
    tweaked_df = clean_cols(df)
    
    tweaked_df = (tweaked_df
                  .assign(month_code=lambda x: pd.Series(
                      x.month.str.split("/")
                      .apply(lambda parts: parts[2] + parts[1])
                      .to_list(), dtype=string_pa),
                          month_code_int=lambda x: x.month_code.astype(int),
                          headcount=pd.Series(dtype=float_pa),
                          num_leaders=pd.Series(dtype=int8_pa)
                         )
                  .query('month_code_int >= @min_month_code', engine='python')
                  .query('business_unit != "Global"')
                  .rename(columns={'budgeted_fte': 'fte_budget',
                                   'average_headcount': 'avg_headcount'})
                  .reset_index()
                  .loc[:, keep_cols]

                 )

    return tweaked_df
    

def tweak_labour(df, report_month):
    """Clean Labour cost dataset.

    Rename columns.
    Set dtypes.
    Remove unwanted columns.
    
    Args:
        df (df): Labour cost data
        report_month (str): Month_Year.

    Returns:
        tweaked_df (df): Cleaned Labour cost data.
    """
    keep_cols = gen_labour_cols(report_month)
    
    tweaked_df = clean_cols(df)
    tweaked_df = (tweaked_df
                  .assign(business_unit=lambda x: x.business_unit.astype(string_pa)
                         )
                  .rename(columns={'business_unit': 'business_unit_jde'})
                  .loc[:, keep_cols]
                 )
    return tweaked_df

### User Input

In [3]:
# User input
month_year = 'October_2025' # Update with month and year being reported on (latest actuals)

### Load and Clean Data

In [4]:
cc_mapping_df = pd.read_csv('Mapping/cost_center_mapping.csv', dtype_backend='pyarrow', engine='pyarrow')
fte_df = pd.read_csv(f'Source/fte_summary_data_full_{month_year}.csv', dtype_backend='pyarrow', engine='pyarrow')
labour_df = pd.read_csv(f'Source/labour_cost_full_{month_year}.csv', dtype_backend='pyarrow', engine='pyarrow')

In [5]:
labour_df = tweak_labour(labour_df, month_year)
cc_mapping_df = tweak_cc_mapping(cc_mapping_df)
fte_df = tweak_fte(fte_df, month_year)

In [6]:
# Add in Workday Business Unit
labour_df = add_wd_bu(labour_df, cc_mapping_df, month_year)

There are entries that do not have a Business Unit attached. Please check the file
File has been saved to Troubleshooting/Missing_Business_Unit_Labour_Costs_October_2025.xlsx


### Generate Labour Costs data
Actual data for most recent month and forecast data for remainder of year.

In [7]:
try:
    labour_cost_df = gen_labour_cost_df(labour_df)
except KeyError as e:
    print(f'Unable to process labour cost file due to {e}')
    print('App will now exit')
    sys.exit()

### Add in FTE Data

In [8]:
org_effect_df = pd.merge(fte_df, labour_cost_df, on=['month_code', 'business_unit'], how='left')

In [9]:
org_effect_df.to_excel(f'Updated Org Effectiveness Data {month_year}.xlsx', index=False)

In [10]:
print('Updated Org Effectiveness file has been generated')

Updated Org Effectiveness file has been generated
